In [1]:
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from datetime import datetime,timedelta
from selenium.webdriver.common.keys import Keys

In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [29]:
dt=pd.read_excel(r"C:\Users\patrick kamga\Liste Communes officielles.xlsx")     # A modifier
dat=dt['Population légale']>=1000
data_f=dt[dat]
nan = dt.index[dt.isnull().any(axis=1)]
dt.drop(nan,0, inplace=True)
colonne=data_f.columns
df_x = pd.DataFrame(data= data_f,columns=colonne)
donnee=df_x.to_numpy()
commune=[]
for i in range(len(donnee)):
    d=donnee[i][2]
    commune.append(d)
vil=data_f['Libellé']
vil=vil.to_numpy()
ville=[]
for i in range(len(vil)):
    ville.append(vil[i])

In [69]:
ville[2701]

'FRENEUSE'

In [3]:
ville=['HERGNIES','ST GEORGES DE RENEINS','FRENEUSE']

In [4]:
driver = webdriver.Firefox(executable_path=r'C:\Users\patrick kamga\geckodriver.exe')     # A modifier
driver.get("https://fr.getaround.com/")
driver.maximize_window()
sleep(5)
accepter=driver.find_element_by_xpath('//*[@class="cobalt-Button cobalt-Button--inversed cobalt-Button--fullWidth"]')
accepter.click()
liste=[]
liste_s=[]
compteur=0
for i in range(len(ville)):
    url='Gare de '+ ville[i]
    #ville='Gare de marseille'
    if compteur==0:
        t=timedelta(days=14)
        #t_demain=timedelta(days=1)
        t_demain=timedelta(days=15)
        #temps=str(datetime.now())[:10]                        # date d'aujourd'hui //à modifier le moment venu
        temps_weeks=str(datetime.now()+t)[:10]               # date dans 02 semaines
        temps_weeks_jour=str(datetime.now()+t_demain)[:10]            # date dans 02 semaines et un jour
        search_bar_ville=driver.find_element_by_xpath('//*[@id="order_address"]')          #barre de recherche des villes
        search_bar_ville.click()
        search_bar_ville.send_keys(url)           #envoie de la requête dans la barre 
        sleep(10)
        ville_search=driver.find_elements_by_xpath('//*[@class="cobalt-Autocomplete__item-value"]')
        #ville_propose=[vil.text for vil in ville_search]
        if len(ville_search)!=1:
            ville_search[1].click()
        else:
            voiture={}
            voiture['ville']=ville[i]
            voiture['commentaire']='Oups ! pas assez précis la recherche'
            active=ville[i]
            liste_s.append(voiture)
            driver.find_element_by_xpath('//*[@class="cobalt-Autocomplete__clear-button"]').click()
            continue
        sleep(2)
        link_date=driver.find_elements_by_xpath('//*[@class="datetime_input_date_wrapper datetime_input js_date_input is_active"]')   #bar date
        link_date[0].click()
        sleep(2)
        date_prev=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day-inner"]')   #date d'aujourd'hui
        #ind=int(temps[8:10])-1
        ind=int(temps_weeks[8:10])-1
        date_prev[ind].click()
        sleep(2)
        time=driver.find_elements_by_xpath('//*[@class="time_picker_entry js_time js_enabled_time"]')
        hr=int(str(datetime.now())[11:13])+1
        if hr<10:
            hr=str(hr)
            indice=[tim.text for tim in time].index('0'+hr+':00')
            time[indice].click()
        else:
            hr=str(hr)
            indice=[tim.text for tim in time].index(hr+':00')
            time[indice].click()
        sleep(2)
        date_next=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day"]')
        last_day=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day cobalt-CalendarRangePicker__day--lastOfMonth"]')
        first_day=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day cobalt-CalendarRangePicker__day--firstOfMonth"]')
        date=[date.get_attribute("data-day") for date in date_next]
        for first in first_day:
            date.append(first.get_attribute("data-day"))
        for last in last_day:
            date.append(last.get_attribute("data-day"))
        sleep(2)
        #indice1=date.index(temps_weeks)
        indice1=date.index(temps_weeks_jour)
        if indice1==len(date)-1:
            last_day.click()
        elif indice1==len(date)-2:
            first_day.click()
        else:
            date_next[indice1].click()
        sleep(2)
        time[indice].click()
        rechercher=driver.find_element_by_xpath('//*[@class="cobalt-Button cobalt-Button--primary js_search_form_submit"]').click()
        sleep(2)
    if compteur==1:
        driver.find_element_by_xpath('//*[@class="cobalt-Autocomplete__clear-button"]').click()
        search_bar=driver.find_element_by_xpath('//*[@id="search_form_address"]')
        search_bar.send_keys(url)
        sleep(10)
        ville_search=driver.find_elements_by_xpath('//*[@class="cobalt-Autocomplete__item-value"]')
        if len(ville_search)!=1:
            ville_search[1].click()
            sleep(5)
        else:
            voiture={}
            voiture['ville']=ville[i]
            voiture['commentaire']='Oups ! pas assez précis la recherche'
            active=ville[i]
            liste_s.append(voiture)
            continue
    compteur=1
    nbre_page=driver.find_elements_by_xpath('//*[@class="cobalt-text-body cobalt-text--subdued"]')
    if len(nbre_page)!=0 and nbre_page[0].text!='':
        nbre=nbre_page[0].text
        n=int(nbre[-2:])
        for j in range(n):
            offers=driver.find_elements_by_xpath('//*[@class="car_card_revamp js_picks_car_card"]')
            for offer in offers:
                offer.click()
                sleep(5)
                nom=driver.find_element_by_xpath('//*[@class="car_info_header__title js_car_name"]')
                distance=driver.find_element_by_xpath('//*[@class="car_info_header__attributes"]')
                prix=driver.find_element_by_xpath('//*[@class="js_price_value"]')
                recuperation=driver.find_element_by_xpath('//*[@class="cobalt-text-titleSmall"]')
                voiture={}
                voiture['ville']=ville[i]
                voiture['nom']=nom.text
                voiture['prix']=prix.text
                voiture['distance']=distance.text[:-13]
                voiture['recuperation']=recuperation.text
                active=ville[i]
                liste.append(voiture)
                driver.find_element_by_xpath('//*[@class="side_panel_close_button js_preview_panel_close hidden-xs"]').click()
                sleep(2)
            suivant=driver.find_elements_by_xpath('//*[@class="cobalt-Button cobalt-Button--ghost cobalt-Button--standalone hidden-xs"]')
            if len(suivant)!=0 and suivant[-1].text =='Suivant':
                suivant[-1].click()
            elif suivant[-1].text =='Précédent'or len(suivant)==0:
                break
            sleep(5)
    else:
        offers=driver.find_elements_by_xpath('//*[@class="car_card_revamp js_picks_car_card"]')
        if len(offers)==0:
            voiture={}
            voiture['ville']=ville[i]
            voiture['commentaire']='Oups ! Aucune voiture en libre-service pour le moment'
            active=ville[i]
            liste_s.append(voiture)
        else:
            for offer in offers:
                sleep(3)
                offer.click()
                sleep(5)
                nom=driver.find_element_by_xpath('//*[@class="car_info_header__title js_car_name"]')
                distance=driver.find_element_by_xpath('//*[@class="car_info_header__attributes"]')
                prix=driver.find_element_by_xpath('//*[@class="js_price_value"]')
                recuperation=driver.find_element_by_xpath('//*[@class="cobalt-text-titleSmall"]')
                voiture={}
                voiture['ville']=ville[i]
                voiture['nom']=nom.text
                voiture['prix']=prix.text
                voiture['distance']=distance.text[:-13]
                voiture['recuperation']=recuperation.text
                active=ville[i]
                liste.append(voiture)
                driver.find_element_by_xpath('//*[@class="side_panel_close_button js_preview_panel_close hidden-xs"]').click()
                sleep(2)

In [5]:
ville.index(active)

2

In [25]:
active

'FRENEUSE'

In [21]:
liste_s

[{'ville': 'HERGNIES', 'commentaire': 'Oups ! pas assez précis la recherche'},
 {'ville': 'FRENEUSE', 'commentaire': 'Oups ! pas assez précis la recherche'}]

In [22]:
liste

[{'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Hyundai iX35',
  'prix': '61€',
  'distance': '6,46 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Renault Twingo',
  'prix': '40€',
  'distance': '8,35 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Peugeot 508',
  'prix': '78€',
  'distance': '5,33 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Mazda 3',
  'prix': '61€',
  'distance': '5,05 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Audi A6',
  'prix': '140€',
  'distance': '8,37 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Opel Corsa',
  'prix': '39€',
  'distance': '5,9 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEORGES DE RENEINS',
  'nom': 'Peugeot 207',
  'prix': '74€',
  'distance': '8,85 km ',
  'recuperation': 'Sur rendez-vous'},
 {'ville': 'ST GEO

In [60]:
v=liste[28]
v['distance'][:-13]

'690 m '

In [8]:
with open('voiture.csv','w') as file:
    file.write('commune,nom,prix,distance,recuperation\n')
    for i in range(len(liste)):
        liste[i]['distance']=re.sub(",",".",liste[i]['distance'])
        file.write(liste[i]['ville']+ ',' +liste[i]['nom'] + ',' + liste[i]['prix'] + ',' + liste[i]['distance'] + ',' + liste[i]['recuperation']+'\n')

In [18]:
driver = webdriver.Firefox(executable_path=r'C:\Users\patrick kamga\geckodriver.exe')     # A modifier
driver.get("https://fr.getaround.com/")
driver.maximize_window()
sleep(5)
accepter=driver.find_element_by_xpath('//*[@class="cobalt-Button cobalt-Button--inversed cobalt-Button--fullWidth"]')
accepter.click()
ville='Gare de paris 20'
liste=[]
t=timedelta(days=14)
t_demain=timedelta(days=15)
#temps=str(datetime.now())[:10]                        # date d'aujourd'hui
temps_weeks=str(datetime.now()+t)[:10]               # date dans 02 semaines
temps_weeks_jour=str(datetime.now()+t_demain)[:10]            # date dans 02 semaines et un jour
search_bar_ville=driver.find_element_by_xpath('//*[@id="order_address"]')          #barre de recherche des villes
search_bar_ville.click()
search_bar_ville.send_keys(ville)
sleep(10)
ville_search=driver.find_elements_by_xpath('//*[@class="cobalt-Autocomplete__item-value"]')
#ville_propose=[vil.text for vil in ville_search]
if len(ville_search)!=0:
    ville_search[1].click()
else:
    ## A revoir
    print(ville)
sleep(5)
link_date=driver.find_elements_by_xpath('//*[@class="datetime_input_date_wrapper datetime_input js_date_input is_active"]')   #bar date
link_date[0].click()
sleep(5)
date_prev=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day-inner"]')   #date d'aujourd'hui
ind=int(temps_weeks[8:10])-1  # à voir
date_prev[ind].click()
sleep(5)
time=driver.find_elements_by_xpath('//*[@class="time_picker_entry js_time js_enabled_time"]')
hr=int(str(datetime.now())[11:13])+1
if hr<10:
    hr=str(hr)
    indice=[tim.text for tim in time].index('0'+hr+':00')
    time[indice].click()
else:
    hr=str(hr)
    indice=[tim.text for tim in time].index(hr+':00')
    time[indice].click()
sleep(5)
date_next=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day"]')
last_day=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day cobalt-CalendarRangePicker__day--lastOfMonth"]')
first_day=driver.find_elements_by_xpath('//*[@class="cobalt-CalendarRangePicker__day cobalt-CalendarRangePicker__day--firstOfMonth"]')
date=[date.get_attribute("data-day") for date in date_next]
for first in first_day:
    date.append(first.get_attribute("data-day"))
for last in last_day:
    date.append(last.get_attribute("data-day"))
sleep(5)
indice1=date.index(temps_weeks_jour)   #à voir
if indice1==len(date)-1:
    last_day.click()
elif indice1==len(date)-2:
    first_day.click()
else:
    date_next[indice1].click()
sleep(5)
time[indice].click()
rechercher=driver.find_element_by_xpath('//*[@class="cobalt-Button cobalt-Button--primary js_search_form_submit"]').click()
#offers=driver.find_elements_by_xpath('//*[@class="car_card_revamp js_picks_car_card"]')

In [17]:
len(date_next)

55

In [15]:
<div class="cobalt-CalendarRangePicker__day cobalt-CalendarRangePicker__day--range cobalt-CalendarRangePicker__day--selected" data-day="2021-08-04"><div class="cobalt-CalendarRangePicker__day-inner">4</div></div>

55

In [24]:
for offer in offers:
                sleep(3)
                offer.click()
                sleep(5)
                nom=driver.find_element_by_xpath('//*[@class="car_info_header__title js_car_name"]')
                distance=driver.find_element_by_xpath('//*[@class="car_info_header__attributes"]')
                prix=driver.find_element_by_xpath('//*[@class="js_price_value"]')
                recuperation=driver.find_element_by_xpath('//*[@class="cobalt-text-titleSmall"]')
                voiture={}
                voiture['ville']=ville[i]
                voiture['nom']=nom.text
                voiture['prix']=prix.text
                voiture['distance']=distance.text[:-13]
                voiture['recuperation']=recuperation.text
                liste.append(voiture)
                driver.find_element_by_xpath('//*[@class="side_panel_close_button js_preview_panel_close hidden-xs"]').click()
                sleep(2)


['GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT',
 'GETAROUND CONNECT']

In [11]:
import time
from datetime import datetime,timedelta
date=str(datetime.now())

'2021-08-03'

In [105]:
V=[[1,2,3],[4,5],[9]]
w=[]
for v in V:
    for i in range(len(v)):
        w.append(v[i])
w

[1, 2, 3, 4, 5, 9]

In [145]:
1 in w

True

In [64]:
int(date[11:13])+1

15

In [85]:
myDateTime1 = datetime(2014, 10, 27)
myDateTime2 = datetime(2014, 10, 23)
td = myDateTime2 - myDateTime1


In [36]:
td

datetime.timedelta(days=14)

In [86]:
t=timedelta(days=14)
myDateTime2=myDateTime1+t

In [156]:
str(myDateTime2)[8:10]

'10'

In [52]:
str(datetime.now()+t)[:10]

'2021-08-10'